In [1]:
import pyspark

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200508150103-0000
KERNEL_ID = 8afa9435-a7c0-42a7-9e78-0a0f9711aa45


In [2]:
import ibmos2spark
cloud_credentials = {
  "apikey": "BBTTv16fkk0Dd6gz9ttr7dq7bsrFsnrUGmmZTIoiw1vD",
  "cos_hmac_keys": {
    "access_key_id": "c0f751954f5642949ba3fc76d6216859",
    "secret_access_key": "88a1ada8161d101261880768719527d0e25f8a69a9d76959"
  },
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "Auto-generated for key c0f75195-4f56-4294-9ba3-fc76d6216859",
  "iam_apikey_name": "WDP-Editor-myfirstsparkproject-donotdelete-pr-a7oeziabdfgdet",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/2629c86ca50d43e39f452ce3799edaf7::serviceid:ServiceId-57d72783-fe0f-4185-a9e1-f31abc5fb319",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/2629c86ca50d43e39f452ce3799edaf7:ede8e02b-407b-4f79-948b-d3f62a1a3634::"
}

credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.softlayer.net',
    'service_id': cloud_credentials['iam_serviceid_crn'],
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token',
    'api_key': cloud_credentials['apikey'],
}

cos = ibmos2spark.CloudObjectStorage(sc, credentials, 'cos_config', 'bluemix_cos')
bucket_name = 'myfirstsparkproject-donotdelete-pr-a7oeziabdfgdet'
csv_name = 'combined-csv-files.csv'

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer, StandardScaler
from pyspark.ml import Pipeline

In [4]:
# Read the CSV File, and Filter Out cancelled flights and rows with no label
flight_data = spark.read.csv(cos.url(csv_name, bucket_name), sep=",", inferSchema="true", header="true")
flight_data = flight_data.filter(flight_data["ARR_TIME"]>0)
flight_data = flight_data.filter(flight_data["ARR_DEL15"]>=0)
flight_data.show(5)

+----+-----+------------+-----------+-------------------+--------------+--------+------+-----------------+------+----------------+---------------+----+--------------+------------+--------+---------+-------------+---------+---------------+--------+----------+---------+-------+------------+--------+---------+-------------+---------+---------------+---------+-----------------+--------+----------------+-------------------+--------+-------+--------+--------------+-------------+-------------+---------+--------------+-------------------+----+
|YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|            FL_DATE|UNIQUE_CARRIER|TAIL_NUM|FL_NUM|ORIGIN_AIRPORT_ID|ORIGIN|ORIGIN_STATE_ABR|DEST_AIRPORT_ID|DEST|DEST_STATE_ABR|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|DEP_DELAY_NEW|DEP_DEL15|DEP_DELAY_GROUP|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|ARR_DELAY_NEW|ARR_DEL15|ARR_DELAY_GROUP|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|FLIGHTS|DISTANCE|DISTANCE_G

In [5]:
# Build a Function To Convert Hours From Military Time 0900, 9AM, to Minutes, 540, to make the predict easier
# Split the String 0900 to 9 hours, and 0 minutes, and do the calculations, this function will take 
# in the dataframe, and the column name to convert, and then return the dataframe with the convert columns added
def convert(flight_data, column_name):
    string_column = flight_data[column_name].cast("string")
    split_col = pyspark.sql.functions.split(string_column, '\d{2}$')[0].cast("int")
    flight_data_convert = flight_data.withColumn(column_name + '_CONVERT', split_col*60 + (flight_data[column_name] - split_col * 100))
    return flight_data_convert

In [6]:
# Running The Function With Converted Columns
list_of_time_column_to_convert = ['CRS_DEP_TIME',
 'CRS_ARR_TIME',
 'ARR_TIME']

for item in list_of_time_column_to_convert:
    flight_data = convert(flight_data,item)

flight_data.show(5)

+----+-----+------------+-----------+-------------------+--------------+--------+------+-----------------+------+----------------+---------------+----+--------------+------------+--------+---------+-------------+---------+---------------+--------+----------+---------+-------+------------+--------+---------+-------------+---------+---------------+---------+-----------------+--------+----------------+-------------------+--------+-------+--------+--------------+-------------+-------------+---------+--------------+-------------------+----+--------------------+--------------------+----------------+
|YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|            FL_DATE|UNIQUE_CARRIER|TAIL_NUM|FL_NUM|ORIGIN_AIRPORT_ID|ORIGIN|ORIGIN_STATE_ABR|DEST_AIRPORT_ID|DEST|DEST_STATE_ABR|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|DEP_DELAY_NEW|DEP_DEL15|DEP_DELAY_GROUP|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|ARR_DELAY_NEW|ARR_DEL15|ARR_DELAY_GROUP|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TI

In [7]:
# In[2]: Build Summary Function
def modelsummary(model):
    import numpy as np
    print ("Note: the last rows are the information for Intercept")
    print ("##","-------------------------------------------------")
    print ("##"," Estimate | Std.Error | t Values | P-value")
    coef = np.append(list(model.coefficients),model.intercept)
    Summary=model.summary
    for i in range(len(Summary.pValues)):
        print ("##",'{:10.6f}'.format(coef[i]),\
        '{:10.6f}'.format(Summary.coefficientStandardErrors[i]),\
        '{:8.3f}'.format(Summary.tValues[i]),\
        '{:10.6f}'.format(Summary.pValues[i]))
    print ("##",'---')
    print ("##","Mean squared error: % .6f" \
    % Summary.meanSquaredError, ", RMSE: % .6f" \
    % Summary.rootMeanSquaredError )
    print ("##","Multiple R-squared: %f" % Summary.r2, ", \
    Total iterations: %i"% Summary.totalIterations)
    

In [8]:
# In[3]: Take Out Necessary Columns, Build One Hot Encoder for  Categorical Columns
"""
Index Out Necessary Columns To Build The Prediction Model
The Categorial Columns Are:
    -- UNIQUE_CARRIER
    -- ORIGIN AIRPORT ID
    -- DEST_AIRPORT_ID
"""
flight_data_select = flight_data.select([
 'UNIQUE_CARRIER',
 'ORIGIN_AIRPORT_ID',
 'DEST_AIRPORT_ID',
 'CRS_DEP_TIME_CONVERT',
 'TAXI_OUT',
 'TAXI_IN',
 'CRS_ARR_TIME_CONVERT',
 'AIR_TIME',
 'DISTANCE',
 'ARR_TIME_CONVERT'])

# Drop Null Data To Prevent Errors
final_data = flight_data_select.na.drop()

carrier_indexer = StringIndexer(inputCol='UNIQUE_CARRIER',outputCol='UNIQUE_CARRIER_INDEX')    
carrier_encoder = OneHotEncoder(inputCol='UNIQUE_CARRIER_INDEX',outputCol='UNIQUE_CARRIER_VEC')

origin_airport_indexer = StringIndexer(inputCol='ORIGIN_AIRPORT_ID',outputCol='ORIGIN_AIRPORT_ID_INDEX')  
origin_airport_encoder = OneHotEncoder(inputCol='ORIGIN_AIRPORT_ID_INDEX',outputCol='ORIGIN_AIRPORT_ID_VEC') 

dest_airport_indexer = StringIndexer(inputCol='DEST_AIRPORT_ID',outputCol='DEST_AIRPORT_ID_INDEX')  
dest_airport_encoder = StringIndexer(inputCol='DEST_AIRPORT_ID_INDEX',outputCol='DEST_AIRPORT_ID_VEC') 


In [9]:
# In[4]: Assembler everything, and build final version including the features.
assembler = VectorAssembler(inputCols=[
 'UNIQUE_CARRIER_VEC',
 'ORIGIN_AIRPORT_ID_VEC',
 'DEST_AIRPORT_ID_VEC',
 'CRS_DEP_TIME_CONVERT',
 'TAXI_OUT',
 'TAXI_IN',
 'CRS_ARR_TIME_CONVERT',
 'AIR_TIME',
 'DISTANCE'],outputCol='features')
                            
ppl = Pipeline(stages=[carrier_indexer,carrier_encoder,
                       origin_airport_indexer, origin_airport_encoder,
                       dest_airport_indexer, dest_airport_encoder,
                       assembler])                   
    
transform_flight_data = ppl.fit(final_data).transform(final_data)
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
scalerModel = scaler.fit(transform_flight_data)
final_transform_data = scalerModel.transform(transform_flight_data)
final_flight_data = final_transform_data.select("features",'ARR_TIME_CONVERT')

In [10]:
# In[5]: Split The Final Flight Data 80-20, and use that to do cross validation

train_data,test_data = final_flight_data.randomSplit([0.8,0.2])

lr = LinearRegression(featuresCol = 'features',labelCol = 'ARR_TIME_CONVERT')

lrModel = lr.fit(final_flight_data)
param_names = [
 'UNIQUE_CARRIER',
 'ORIGIN_AIRPORT_ID',
 'DEST_AIRPORT_ID',
 'CRS_DEP_TIME_CONVERT',
 'TAXI_OUT',
 'TAXI_IN',
 'CRS_ARR_TIME_CONVERT',
 'AIR_TIME',
 'DISTANCE']

modelsummary(lrModel)

Note: the last rows are the information for Intercept
## -------------------------------------------------
##  Estimate | Std.Error | t Values | P-value
##  16.576937   0.512667   32.335   0.000000
##  15.444114   0.527632   29.271   0.000000
##  12.265967   0.517952   23.682   0.000000
##   8.871202   0.524985   16.898   0.000000
##  13.213917   0.513907   25.713   0.000000
##  14.294954   0.520739   27.451   0.000000
##  12.255538   0.533062   22.991   0.000000
##  10.860272   0.535003   20.299   0.000000
##  10.154248   0.559249   18.157   0.000000
##  17.521046   0.560614   31.253   0.000000
##  10.415107   0.610060   17.072   0.000000
##  13.274825   0.599416   22.146   0.000000
##  11.866638   0.642580   18.467   0.000000
##  25.746701   0.811340   31.734   0.000000
## -19.239249  34.962537   -0.550   0.582126
## -13.316789  34.962603   -0.381   0.703287
## -10.153523  34.962993   -0.290   0.771504
##  -8.367375  34.962918   -0.239   0.810856
## -10.857815  34.962967   -0.311   0

## -17.884393  35.025738   -0.511   0.609626
## -15.034006  35.030634   -0.429   0.667801
## -10.292066  35.033348   -0.294   0.768927
## -21.964035  35.033537   -0.627   0.530697
## -20.254737  35.035413   -0.578   0.563182
## -14.405764  35.035687   -0.411   0.680945
##  -8.695656  35.037831   -0.248   0.803996
##  -4.424388  35.039860   -0.126   0.899520
## -13.693210  35.038226   -0.391   0.695939
## -23.291486  35.040879   -0.665   0.506246
## -15.763311  35.041021   -0.450   0.652816
## -24.391666  35.041897   -0.696   0.486384
## -19.931324  35.042027   -0.569   0.569503
## -20.070781  35.043792   -0.573   0.566825
## -21.040894  35.045688   -0.600   0.548250
## -19.193674  35.045218   -0.548   0.583910
## -15.603251  35.046991   -0.445   0.656168
## -18.591966  35.047732   -0.530   0.595782
## -11.230418  35.048409   -0.320   0.748646
## -13.928719  35.048601   -0.397   0.691064
## -14.061017  35.051319   -0.401   0.688306
## -21.559794  35.053127   -0.615   0.538515
##  -5.975